# Prescriptive Models and Data Analytics - Problem Set 2
# Causal Interpretation
## Elen Sahakyan (UID: 706310696)

In [58]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import statsmodels.formula.api as smf   
from scipy import stats
from linearmodels import PanelOLS

**Part 1. Hospital admission & quality of service**

**Question 1. Regressing patient-died dummy variable on a set of hospital dummies**

In [59]:
# Load data
health_df = pd.read_csv('health_data.csv')
health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24480 entries, 0 to 24479
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   patient_id          24480 non-null  int64 
 1   hospital_id         24480 non-null  object
 2   admin_year          24480 non-null  int64 
 3   patient_died_dummy  24480 non-null  int64 
 4   startage            24480 non-null  int64 
 5   female_dummy        24480 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ MB


In [60]:
health_df.describe()

,patient_id,admin_year,patient_died_dummy,startage,female_dummy
count,24480.000000,24480.000000,24480.000000,24480.000000,24480.000000
mean,12240.500000,2004.882271,0.128554,65.465523,0.201389
std,7066.911631,1.428537,0.334712,9.056334,0.401046
min,1.000000,2003.000000,0.000000,25.000000,0.000000
25%,6120.750000,2004.000000,0.000000,59.000000,0.000000
50%,12240.500000,2005.000000,0.000000,66.000000,0.000000
75%,18360.250000,2006.000000,0.000000,72.000000,0.000000
max,24480.000000,2007.000000,1.000000,89.000000,1.000000


In [61]:
health_df.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
0,1,D,2003,0,81,0
1,2,H,2003,1,67,0
2,3,A,2003,0,54,0
3,4,E,2003,0,81,0
4,5,H,2003,0,69,0


In [62]:
# Regress patient-died dummy variable on a set of hospital dummies
# Exogenous variables of patient characteristics are not included in this analysis
reg_hospital_ids = smf.ols(formula = 'patient_died_dummy ~ C(hospital_id)', data = health_df).fit()   
print(reg_hospital_ids.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     119.3
Date:                Sun, 11 Feb 2024   Prob (F-statistic):          1.75e-220
Time:                        17:50:30   Log-Likelihood:                -7416.5
No. Observations:               24480   AIC:                         1.485e+04
Df Residuals:                   24470   BIC:                         1.493e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0970    

The constant term has a coefficient of 0.0970, which implies that keeping other variables at their reference levels, including keeping the variable `hospital_id` at baseline, i.e. Hospital A, the estimated probability of patient death is 0.0970 (9.7%).

The dummy variable coefficient for Hospital D is 0.1882, indicating that the estimated probability of patient death in hospital D is 0.1882 (18.82%) higher as compared to Hospital A, the baseline hospital, holding all other variables constant.

In [63]:
# Difference in the mortality rates, i.e. probability of patient death, in Hospitals D and E
coef_hospital_D = reg_hospital_ids.params['C(hospital_id)[T.D]']
coef_hospital_E = reg_hospital_ids.params['C(hospital_id)[T.E]']
diff_D_E = coef_hospital_D - coef_hospital_E

print(f'Difference in mortality rates at Hospitals D and E is {diff_D_E:.4f}.') 

Difference in mortality rates at Hospitals D and E is 0.2414.


**Question 2. Mortality rates in Hospitals A and B**

In [64]:
# Use only the data from Hospitals A and B
# Regress mortality on a dummy for whether the patient visited hospital B
health_df_A_B = health_df[health_df['hospital_id'].isin(['A', 'B'])]
reg_B = smf.ols(formula = 'patient_died_dummy ~ C(hospital_id)', data = health_df_A_B).fit()   

print(reg_B.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9377
Date:                Sun, 11 Feb 2024   Prob (F-statistic):              0.333
Time:                        17:50:30   Log-Likelihood:                -1446.8
No. Observations:                6611   AIC:                             2898.
Df Residuals:                    6609   BIC:                             2911.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0970    

a) The coefficient for `hospital_id` is 0.0072 (0.72%), with P value higher than the significance level of 0.05. The coefficient indicates that patients admitted at Hospital B have 0.72% higher probability of death as compared to the baseline category of Hospital A. However, as the P value is large, we do not have sufficient evidence to claim that the coefficient is significant, and thus, cannot claim that there is significant difference in mortality rates between Hospitals A and B.

b) Hospital admission is not entirely random and depends on several factors including proximity, or specialization. As Hospitals usually specialize in certain treatments, if the patients exhibit symptoms of (CABG), the emergency might take them to a specialized hospital. Moreover, some hospitals may have specialized gynecology or maternity departments, thus, female patients may prefer or be admitted to one hospital over the other. 

Taking these factors into account, I believe there are omitted variables present, including the gender of the patient and age, which might cause an underestimation in the Hospital_id coefficient, given that the correlation between the gender and hospital_id is negative. The patient gender is controlled next. 

In [65]:
# Control for patient age to obtain a causal estimate
reg_B_age = smf.ols(formula = 'patient_died_dummy ~ C(hospital_id) + female_dummy + startage', data = health_df_A_B).fit()   

print(reg_B_age.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     147.6
Date:                Sun, 11 Feb 2024   Prob (F-statistic):           1.43e-92
Time:                        17:50:30   Log-Likelihood:                -1232.8
No. Observations:                6611   AIC:                             2474.
Df Residuals:                    6607   BIC:                             2501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.1165    

c) Including the dummy variable for gender and age, increased the coefficient of the dummy variable `hospital_id` to 0.0114 (1.14%). This indicates that there is positive relationship between the patient characteristics, `female_dummy` and `startage`, and the likelihood of admission to Hospital B as compared to Hospital A. 

The P-value for `hospital_id` is a little above the significance level of 10%, which is an improvement to the previous regression model. Controlling for the gender of the patient, we have more confidence to claim that patients admitted to Hospital B have 1.21% higher mortality rate as compared to patients in Hospital A.

**Part 2. Demand estimation**

**Question 1. Regress sales on price, and another regression on price and a summer dummy for Vendor 1.**

In [66]:
# Load data
demand_df = pd.read_csv('demand_data.csv')
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   vendor_id     5200 non-null   int64  
 1   week          5200 non-null   int64  
 2   summer_dummy  5200 non-null   int64  
 3   price         5200 non-null   float64
 4   sales         5200 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 203.3 KB


In [67]:
demand_df.describe()

,vendor_id,week,summer_dummy,price,sales
count,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000
mean,50.500000,26.500000,0.250000,3.292019,8834.468677
std,28.868846,15.009774,0.433054,1.207849,545.338400
min,1.000000,1.000000,0.000000,1.500000,3249.217300
25%,25.750000,13.750000,0.000000,2.500000,8705.932375
50%,50.500000,26.500000,0.000000,3.000000,8875.519500
75%,75.250000,39.250000,0.250000,4.000000,9051.101825
max,100.000000,52.000000,1.000000,6.000000,9851.132800


In [68]:
demand_df.head()    

,vendor_id,week,summer_dummy,price,sales
0,1,1,0,2.0,8788.7383
1,1,2,0,3.0,8937.9863
2,1,3,0,3.0,8740.1777
3,1,4,0,3.0,8757.1338
4,1,5,0,3.0,8739.6104


In [69]:
# Subset demand data for the Vendor 1, make sure data contains only 52 weeks
demand_v1 = demand_df[demand_df['vendor_id'] == 1]
demand_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 51
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   vendor_id     52 non-null     int64  
 1   week          52 non-null     int64  
 2   summer_dummy  52 non-null     int64  
 3   price         52 non-null     float64
 4   sales         52 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 2.4 KB


In [70]:
# Regress sales on price for Vendor 1
reg_v1 = smf.ols(formula = 'sales ~ price', data = demand_v1).fit()
print(reg_v1.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.3250
Date:                Sun, 11 Feb 2024   Prob (F-statistic):              0.571
Time:                        17:50:31   Log-Likelihood:                -360.33
No. Observations:                  52   AIC:                             724.7
Df Residuals:                      50   BIC:                             728.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8983.8227    145.437     61.771      0.0

In [71]:
# Regress sales on price and summer dummy for Vendor 1
reg_v1_summer = smf.ols(formula = 'sales ~ price + summer_dummy', data = demand_v1).fit()
print(reg_v1_summer.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     11.42
Date:                Sun, 11 Feb 2024   Prob (F-statistic):           8.49e-05
Time:                        17:50:31   Log-Likelihood:                -350.56
No. Observations:                  52   AIC:                             707.1
Df Residuals:                      49   BIC:                             713.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     9177.5500    128.432     71.458   

After including the `summer_dummy` in the regression, we can notice that the `price` coefficient decreases, from the initial `-31.23` to `-141.19`. Using the formula for the omitted variable bias, we can infer that the covariance between the two independent variables, `price` and `summer_dummy` is negative, which indicates that the price decreases during the summer months. 

**Question 2. Regress sales on price, and another regression on price and a summer dummy for Vendor 2.**

In [72]:
# Subset demand data for the Vendor 2
# Regress sales on price for Vendor 2
demand_v2 = demand_df[demand_df['vendor_id'] == 2]

reg_v2 = smf.ols(formula = 'sales ~ price', data = demand_v2).fit()
print(reg_v2.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Sun, 11 Feb 2024   Prob (F-statistic):            0.00781
Time:                        17:50:31   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8411.1748    219.545     38.312      0.0

In [73]:
# Regress sales on price and summer dummy for Vendor 2
reg_v2_summer = smf.ols(formula = 'sales ~ price + summer_dummy', data = demand_v2).fit()
print(reg_v2_summer.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Sun, 11 Feb 2024   Prob (F-statistic):            0.00781
Time:                        17:50:31   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2105.3159     29.848     70.534   

In [57]:
# Check for correlation between independent variables
demand_v2.corr()

,vendor_id,week,summer_dummy,price,sales
vendor_id,NaN,NaN,NaN,NaN,NaN
week,NaN,1.000000,0.057703,0.057703,0.152865
summer_dummy,NaN,0.057703,1.000000,1.000000,0.364969
price,NaN,0.057703,1.000000,1.000000,0.364969
sales,NaN,0.152865,0.364969,0.364969,1.000000


The note indicates that there is a possible multicollinearity present, which was confirmed by the correlation matrix: `price` is perfectly correlated with `summer_dummy`, price is artificially increased during summer by Vendor 2.

Although the practice and literature allows for some multicollinearity between independent variables, perfect multicollinearity affects the precision of the coefficients. 

**Question 3. Suppose price is not systematically increased/decreased during summer, what would happen to regression coefficients?**

Assuming that ice-cream vendors are in manageable proximity from one another, and one vendor's prices affect also the prices of other vendors, and, assuming that Vendor 2 does not systematically increase price during the summer weeks, we can state that:

- *For Vendor 1:* The coefficient of `summer_dummy` will decrease, as there will be more organic competition from Vendor 2. `price` will have higher negative effect on the sales as well, i.e. the coefficient will decrease.

- *For Vendor 2:* The coefficient of `summer_dummy` will increase as it is likely to be positive, since people consume more ice-cream during summer. However, the coefficient for the `price` will increase. If we look at the OVB formula, price and seasonality have positive covariance (demand during summer months pushes price to increase) and the `summer_dummy` has a positive effect on ice-cream demand. 